In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import os
import numpy as np

In [61]:
data = []
labels = []
classes = 43
cwd = 'G:\\Deep Learning\\Traffic Sign Detection'
for i in range(classes):
    path = os.path.join(cwd, 'train', str(i))
    images = os.listdir(path)
    for img in images:
        try:
            image = Image.open(path + "\\" + img)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print('Error loading image.')
data = np.array(data)
labels = np.array(labels)

In [62]:
print(len(data))
print(len(labels))
print('Train data shape: {}  Labels shape: {}'.format(data.shape, labels.shape))

39209
39209
Train data shape: (39209, 30, 30, 3)  Labels shape: (39209,)


In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    
print('X_train {},   y_train {}'.format(X_train.shape, y_train.shape))
print('X_test {},  y_test {}'.format(X_test.shape, y_test.shape))

X_train (31367, 30, 30, 3),   y_train (31367,)
X_test (7842, 30, 30, 3),  y_test (7842,)


In [64]:
y_train = tf.keras.utils.to_categorical(y_train, 43)
y_test = tf.keras.utils.to_categorical(y_test, 43)

In [65]:
X_train.shape[1:]

(30, 30, 3)

### Building the Model

In [66]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5,5), activation='relu', input_shape= X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(32, kernel_size=(5,5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.50))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [67]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [68]:
EPOCHS = 15
BATCH_SIZE = 64

In [69]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

Train on 31367 samples, validate on 7842 samples
Epoch 1/15
31367/31367 [==============================] - 148s 5ms/sample - loss: 2.2202 - accuracy: 0.4627 - val_loss: 0.5813 - val_accuracy: 0.8416
Epoch 2/15
31367/31367 [==============================] - 140s 4ms/sample - loss: 0.6983 - accuracy: 0.7983 - val_loss: 0.2376 - val_accuracy: 0.9308
Epoch 3/15
31367/31367 [==============================] - 145s 5ms/sample - loss: 0.4353 - accuracy: 0.8752 - val_loss: 0.1523 - val_accuracy: 0.9551
Epoch 4/15
31367/31367 [==============================] - 142s 5ms/sample - loss: 0.3237 - accuracy: 0.9043 - val_loss: 0.1122 - val_accuracy: 0.9684
Epoch 5/15
31367/31367 [==============================] - 142s 5ms/sample - loss: 0.2858 - accuracy: 0.9175 - val_loss: 0.0870 - val_accuracy: 0.9739
Epoch 6/15
31367/31367 [==============================] - 141s 4ms/sample - loss: 0.2450 - accuracy: 0.9307 - val_loss: 0.0784 - val_accuracy: 0.9753
Epoch 7/15
31367/31367 [===========================

In [70]:
model.evaluate(X_test, y_test)

7842/7842 [==============================] - 10s 1ms/sample - loss: 0.0494 - accuracy: 0.9850


[0.049435986130507495, 0.9849528]

In [77]:
model.save('model.h5')

In [78]:
y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
pred = model.predict_classes(X_test)
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

0.9533650039588282


In [79]:
model.save('traffic_signal_classifier.h5')